Here's a practical approach along with the Python code to define a **Customer Utility Function** based on **Cumulative Prospect Theory (CPT)**, tailored specifically to your dataset:

---

## **Step 2: Defining Customer Utility Functions (CPT-based)**

### **Objective**
Create a mathematical utility function reflecting customer decision-making, incorporating **loss aversion** and probability weighting (key aspects of CPT).

### **Theoretical Background:**
According to CPT, utility depends on perceived gains or losses relative to a **reference point** (in this case, the customer's current satisfaction level).

The general CPT utility formula:

$$
U(Customer) = w^+(P_{stay}) \times V(\text{Gain}) - w^-(P_{churn}) \times V(\text{Loss})
$$

- $w^+(·), w^-(·)$: Probability weighting functions (customers overweight low probabilities, underweight high probabilities).
- $V(x)$: Value function, typically reflecting loss aversion (losses loom larger than equivalent gains).

---

## **Defining Parameters Practically:**

- **Reference point (r)**: Customer’s current satisfaction (`Satisfaction_Score`)
- **Gain**: Increment in satisfaction or benefit from staying (e.g., personalized promotions)
- **Loss**: The potential dissatisfaction or perceived negative impact of leaving (e.g., losing accumulated loyalty benefits, personalized support)

---

## **Implementation with Python:**

Here's practical code to define the utility function clearly in Python:

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('online_retail_customer_churn.csv')

# Parameters based on typical Prospect Theory values (can be calibrated later)
alpha = 0.88      # sensitivity parameter for gains
beta = 0.88       # sensitivity parameter for losses
lambda_ = 2.25    # loss aversion coefficient (typical range: 2.0 - 2.5)
gamma = 0.61      # parameter for probability weighting function

# Probability weighting function based on CPT
def weighting_function(p, gamma):
    return np.exp(-(-np.log(p))**gamma)

# Calculate normalized satisfaction score (reference point)
data['Normalized_Satisfaction'] = data['Satisfaction_Score'] / data['Satisfaction_Score'].max()

# Estimate probability of churn from data
churn_prob = data['Target_Churn'].mean()
stay_prob = 1 - churn_prob

# Compute value functions for Gain and Loss based on Satisfaction_Score
data['V_Gain'] = (data['Normalized_Satisfaction'] ** alpha)
data['V_Loss'] = lambda_ * ((1 - data['Normalized_Satisfaction']) ** beta)

# Apply probability weighting
data['w_P_stay'] = weighting_function(stay_prob, gamma)
data['w_P_churn'] = weighting_function(churn_prob, gamma)

# Calculate Customer Utility based on CPT formula
data['Customer_Utility'] = (data['w_P_stay'] * data['V_Gain']) - (data['w_P_churn'] * data['V_Loss'])

# Display the results clearly (first 10 customers)
display_columns = [
    'Customer_ID',
    'Satisfaction_Score',
    'Normalized_Satisfaction',
    'V_Gain',
    'V_Loss',
    'w_P_stay',
    'w_P_churn',
    'Customer_Utility'
]

# Displaying the calculated utilities
print("Customer Utility Function based on CPT (first 10 rows):")
display(data[display_columns].head(10))


Customer Utility Function based on CPT (first 10 rows):


,Customer_ID,Satisfaction_Score,Normalized_Satisfaction,V_Gain,V_Loss,w_P_stay,w_P_churn,Customer_Utility
0,1,3,0.6,0.637930,1.004605,0.433141,0.466053,-0.191885
1,2,3,0.6,0.637930,1.004605,0.433141,0.466053,-0.191885
2,3,2,0.4,0.446491,1.435343,0.433141,0.466053,-0.475552
3,4,5,1.0,1.000000,0.000000,0.433141,0.466053,0.433141
4,5,5,1.0,1.000000,0.000000,0.433141,0.466053,0.433141
5,6,4,0.8,0.821711,0.545870,0.433141,0.466053,0.101513
6,7,1,0.2,0.242609,1.848850,0.433141,0.466053,-0.756579
7,8,3,0.6,0.637930,1.004605,0.433141,0.466053,-0.191885
8,9,4,0.8,0.821711,0.545870,0.433141,0.466053,0.101513
9,10,2,0.4,0.446491,1.435343,0.433141,0.466053,-0.475552


The calculated **Customer Utility** values represent how customers perceive the utility (value) of staying with your store based on their satisfaction and churn probabilities.

- **Positive Utility Values**:  
  - Customers perceive the overall benefit (**gain**) from remaining as greater than the **loss** they'd incur by leaving.
  - Indicates customers are more likely to continue purchasing and remain loyal, making them less at risk of churn.
  - **Example from data**: Customers 4 and 5 (high normalized satisfaction = 1) have positive utility, indicating they're satisfied and perceive high value from their relationship with your business.

  - **Practical Implication for Business**:
    - Low-priority targets for retention interventions (already likely to stay).


- **Negative Utility Values**:
  - Occur when customers perceive the **loss** from continuing the relationship (such as dissatisfaction, frustration, poor service) to be greater than any perceived benefit (**gain**).
  - Indicates these customers are more at risk of churn and may require targeted interventions or offers to remain loyal.
  - The magnitude of negativity indicates urgency: the more negative, the higher the churn risk.
  - **Example from data**: Customers 0, 1, 2, 6, and 7 show negative utility, indicating dissatisfaction or risk of leaving.

---

### **How to practically use these insights?**

- **Customers with Positive Utility**:
  - Minimal or standard retention efforts (maintain good experience).
- **Customers with negative utility**:
  - Target proactively with strategic interventions.
  - Address dissatisfaction immediately (personalized support, discounts, loyalty rewards).

This utility metric is invaluable in prioritizing retention efforts efficiently and maximizing overall business impact.